# Tugas Besar IF3170 Inteligensia Buatan 2017/2018

## Aplikasi Web Prediksi Income Per Tahun Tahap 1 : Eksperimen

Anggota :
    1. Dewita Sonya Tarabunga - 13515021
    2. Helena Suzane Graciella - 13515032
    4. Emilia Razak - 13515056
    3. Jehian Norman Saviero - 13515139

### Analisis Data

Data training yang diberikan merupakan data dari 32561 hasil sensus yang memberikan data-data berupa age, workclass, final weight, education, education number, marital status, occupation, race, sex, capital gain, capital loss, hours per week, dan native country. Terdapat juga data test yang akan digunakan untuk melakukan test hasil pembelajaran.

Analisis yang kami lakukan terhadap data adalah analisis terhadap missing value pada data. Terdapat 3 kolom pada data yang memiliki missing value, yaitu '?' yaitu workclass, occupation, dan native country. Selain itu, semua kolom lain tidak memiliki missing value. Setelah dilihat dari data, semua orang yang memiliki missing value pada occupation hanya memiliki dua kemungkinan nilai workclass, yaitu Never-worked atau missing value. Jelas bahwa jika workclass adalah Never-worked, maka orang tersebut tidak memiliki pekerjaan sehingga value nya kosong. Lalu, jika dilihat dari data yang memiliki missing value di keduanya, hampir 90% dari orang tersebut memiliki gross income <= 50K, maka kami mengasumsikan bahwa orang tersebut pernah bekerja, namun sekarang tidak bekerja sehingga tidak masuk ke kategori manapun dalam workclass. Maka kami mengubah missing value di workclass menjadi Not-worked dan missing value di occupation menjadi None.

Satu kolom lagi yang memiliki missing value adalah native country, namun native country merupakan data categorical sehingga tidak masuk akal jika diganti dengan data mean atau median, begitu pula data modus juga kurang cocok untuk menggantikan missing value. Maka kami menganggap missing value sebagai negara lain di luar daftar, atau Other
Setelah analisis, kami akan melakukan eksperimen dengan akurasi menjadi ukuran kinerja.

### Eksperimen : 10-Cross Fold Validation

#### Import library yang dibutuhkan

In [19]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

#### Membaca data training dari file eksternal

In [2]:
data = pd.read_csv('CencusIncome.data.txt')
print(data.head())

   age   workclass  fnlwgt        edu  edunum             marital occupation  \
0   42  Not-worked   51795    HS-grad       9            Divorced       None   
1   60  Not-worked  251572    HS-grad       9             Widowed       None   
2   28  Not-worked  201844    HS-grad       9           Separated       None   
3   28  Not-worked  196630  Assoc-voc      11           Separated       None   
4   67  Not-worked  150264  Doctorate      16  Married-civ-spouse       None   

    relationship   race     sex  gain  loss  hours         native  gross  
0      Unmarried  Black  Female     0     0     32  United-States  <=50K  
1  Not-in-family  White    Male     0     0     35         Poland  <=50K  
2      Unmarried  White  Female     0     0     40         Mexico  <=50K  
3      Unmarried  White  Female     0     0     40         Mexico  <=50K  
4        Husband  White    Male     0     0     20         Canada   >50K  


#### Melakukan encoding pada data categorical

In [3]:
# Digunakan OneHotEncoder pada data categorical selain gross income
for column in data.columns.values:
    if (data.dtypes[column] == object and column != 'gross'):
        enc = pd.get_dummies(data[column])
        data = data.drop(column, axis=1)
        data = data.join(enc)
# Digunakan LabelEncoder pada data gross karena kolom tersebut merupakan target
le = preprocessing.LabelEncoder()
le.fit(['<=50K', '>50K'])
enc = le.transform(data['gross'])
data['gross'] = enc
print(data.head())

   age  fnlwgt  edunum  gain  loss  hours  gross  Federal-gov  Local-gov  \
0   42   51795       9     0     0     32      0            0          0   
1   60  251572       9     0     0     35      0            0          0   
2   28  201844       9     0     0     40      0            0          0   
3   28  196630      11     0     0     40      0            0          0   
4   67  150264      16     0     0     20      1            0          0   

   Never-worked     ...      Portugal  Puerto-Rico  Scotland  South  Taiwan  \
0             0     ...             0            0         0      0       0   
1             0     ...             0            0         0      0       0   
2             0     ...             0            0         0      0       0   
3             0     ...             0            0         0      0       0   
4             0     ...             0            0         0      0       0   

   Thailand  Trinadad&Tobago  United-States  Vietnam  Yugoslavia  
0

#### Melakukan split antara data dan target untuk difit

In [4]:
datatrain = data.drop('gross', axis=1)
targettrain = data['gross']

#### Import Confusion Matrix

In [5]:
from sklearn.metrics import confusion_matrix

#### Melakukan eksperimen dengan Naive Bayes

In [22]:
gnb = GaussianNB()
for train_index, test_index in kf.split(datatrain):
        X_train, X_test = datatrain.iloc[train_index], datatrain.iloc[test_index]
        Y_train, Y_test = targettrain.iloc[train_index], targettrain.iloc[test_index]
        gnb = gnb.fit(X_train, Y_train)
        pred = gnb.predict(X_test)
        print(accuracy_score(Y_test, pred))
        print(confusion_matrix(Y_test, pred, labels=[0,1]))

0.78999078907
[[2330  141]
 [ 543  243]]
0.798832923833
[[2356  120]
 [ 535  245]]
0.795147420147
[[2366  120]
 [ 547  223]]
0.793304668305
[[2335  119]
 [ 554  248]]
0.781326781327
[[2301  143]
 [ 569  243]]
0.795454545455
[[2355  131]
 [ 535  235]]
0.790540540541
[[2327  139]
 [ 543  247]]
0.800061425061
[[2346  125]
 [ 526  259]]
0.79699017199
[[2355  115]
 [ 546  240]]
0.810503685504
[[2383  113]
 [ 504  256]]


#### Melakukan ekperimen dengan Decision Tree Classifier

In [16]:
kf = KFold(n_splits=10, shuffle=True)
tclf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=10)
for train_index, test_index in kf.split(datatrain):
        X_train, X_test = datatrain.iloc[train_index], datatrain.iloc[test_index]
        Y_train, Y_test = targettrain.iloc[train_index], targettrain.iloc[test_index]
        tclf = tclf.fit(X_train, Y_train)
        pred = tclf.predict(X_test)
        print(accuracy_score(Y_test, pred))
        print(confusion_matrix(Y_test, pred, labels=[0,1]))

0.851089960086
[[2313  151]
 [ 334  459]]
0.85042997543
[[2296  156]
 [ 331  473]]
0.84613022113
[[2332  147]
 [ 354  423]]
0.859029484029
[[2290  210]
 [ 249  507]]
0.862407862408
[[2338  123]
 [ 325  470]]
0.848894348894
[[2293  159]
 [ 333  471]]
0.853501228501
[[2302  183]
 [ 294  477]]
0.865479115479
[[2322  110]
 [ 328  496]]
0.856265356265
[[2380  116]
 [ 352  408]]
0.860872235872
[[2413   86]
 [ 367  390]]


#### Melakukan ekperimen dengan Multilayer Perceptron

In [17]:
nclf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(20, 20, 20), random_state=1, max_iter=500)
for train_index, test_index in kf.split(datatrain):
        X_train, X_test = datatrain.iloc[train_index], datatrain.iloc[test_index]
        Y_train, Y_test = targettrain.iloc[train_index], targettrain.iloc[test_index]
        nclf = nclf.fit(X_train, Y_train)
        pred = nclf.predict(X_test)
        print(accuracy_score(Y_test, pred))
        print(confusion_matrix(Y_test, pred, labels=[0,1]))

0.792140006141
[[2397   71]
 [ 606  183]]
0.790847665848
[[2469   17]
 [ 664  106]]
0.24785012285
[[   0 2449]
 [   0  807]]
0.240786240786
[[   0 2472]
 [   0  784]]
0.777641277641
[[2211  282]
 [ 442  321]]
0.782555282555
[[2435   14]
 [ 694  113]]
0.797911547912
[[2367  132]
 [ 526  231]]
0.765663390663
[[2468    3]
 [ 760   25]]
0.789926289926
[[2408   66]
 [ 618  164]]
0.244778869779
[[   0 2459]
 [   0  797]]


### Full Training

#### Training

In [8]:
tclf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=10)
tclf = tclf.fit(datatrain, targettrain)

#### Simpan model ke file eksternal

In [9]:
joblib.dump(tclf, 'model.pkl')

['model.pkl']

### Evaluasi Model

#### Load model dari file eksternal

In [10]:
model = joblib.load('model.pkl')

#### Membaca data test dari file eksternal

In [11]:
test = pd.read_csv('CencusIncome.test.txt')
print(test.head())

   age   workclass  fnlwgt           edu  edunum             marital  \
0   25     Private  226802          11th       7       Never-married   
1   38     Private   89814       HS-grad       9  Married-civ-spouse   
2   28   Local-gov  336951    Assoc-acdm      12  Married-civ-spouse   
3   44     Private  160323  Some-college      10  Married-civ-spouse   
4   58  Not-worked  299831       HS-grad       9  Married-civ-spouse   

          occupation relationship   race   sex  gain  loss  hours  \
0  Machine-op-inspct    Own-child  Black  Male     0     0     40   
1    Farming-fishing      Husband  White  Male     0     0     50   
2    Protective-serv      Husband  White  Male     0     0     40   
3  Machine-op-inspct      Husband  Black  Male  7688     0     40   
4               None      Husband  White  Male     0     0     35   

          native  gross  
0  United-States  <=50K  
1  United-States  <=50K  
2  United-States   >50K  
3  United-States   >50K  
4  United-States  <=50

#### Melakukan encoding pada data categorical

In [12]:
# Digunakan OneHotEncoder pada data categorical selain gross income
for column in test.columns.values:
    if (test.dtypes[column] == object and column != 'gross'):
        enc = pd.get_dummies(test[column])
        test = test.drop(column, axis=1)
        test = test.join(enc)
# Digunakan LabelEncoder pada data gross karena kolom tersebut merupakan target
le = preprocessing.LabelEncoder()
le.fit(['<=50K', '>50K'])
enc = le.transform(test['gross'])
test['gross'] = enc
print(test.head())

   age  fnlwgt  edunum  gain  loss  hours  gross  Federal-gov  Local-gov  \
0   25  226802       7     0     0     40      0            0          0   
1   38   89814       9     0     0     50      0            0          0   
2   28  336951      12     0     0     40      1            0          1   
3   44  160323      10  7688     0     40      1            0          0   
4   58  299831       9     0     0     35      0            0          0   

   Never-worked     ...      Portugal  Puerto-Rico  Scotland  South  Taiwan  \
0             0     ...             0            0         0      0       0   
1             0     ...             0            0         0      0       0   
2             0     ...             0            0         0      0       0   
3             0     ...             0            0         0      0       0   
4             0     ...             0            0         0      0       0   

   Thailand  Trinadad&Tobago  United-States  Vietnam  Yugoslavia  
0

#### Menambah kolom yang kurang pada data test

Terdapat satu kategori pada native country yang tidak muncul pada data test, yaitu Holand-Netherlands. Akibatnya saat dilakukan OneHotEncoding, kolom tersebut tidak muncul, padahal kolom dari data training dan test harus sama. Maka solusinya adalah menambah kolom Holand-Netherlands yang berisi 0 pada setiap row.

In [13]:
test['Holand-Netherlands'] = 0
test = test[data.columns.tolist()]

#### Melakukan split antara data dan target untuk diprediksi

In [14]:
datatest = test.drop('gross', axis=1)
targettest = test['gross']

#### Melakukan prediksi

In [15]:
pred = model.predict(datatest)
print(pred)
print(accuracy_score(targettest, pred))

[0 0 0 ..., 0 0 0]
0.860020883238
